In [1]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf
from tensorflow.contrib import layers
import numpy as np

env = gym.make("FrozenLake-v0")


hdf5 not supported (please install/reinstall h5py)


[2017-01-25 16:46:04,936] Making new env: FrozenLake-v0


In [27]:
n_actions = env.action_space.n
n_states = env.observation_space.n
learning_rate = 0.85
y = 0.9
model_name = "deep-policy-gradient.model"
model_path = "/models/" + model_name

graph = tf.Graph()
gsess = tf.InteractiveSession(graph=graph)
with graph.as_default():
    with tf.device("cpu:0"):
        s = tf.placeholder(tf.int32, ())
        step = tf.placeholder(tf.int32, ())
        a = tf.placeholder(tf.int32, ())
        r = tf.placeholder(tf.float32, ())
        

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))


        Ps = Pipe(
            s,
            T.one_hot(n_states).expand_dims(0)
            .relu_layer(32, scope='relu_layer', variables_collections=["actor"], **ops)
            .softmax_layer(n_actions, scope='softmax_layer', variables_collections=["actor"], **ops),
            T[0]
        )
        Psa = Ps[a]
        
        V = Pipe(
            s,
            T.one_hot(n_states).expand_dims(0)
            .relu_layer(32, scope='relu_layer_critic', variables_collections=["critic"], **ops)
            .linear_layer(1, scope='linear_layer_critic', variables_collections=["critic"], **ops),
            T[0, 0]
        )
        
        ws_actor = tf.get_collection("actor")
        gradients_actor = tf.gradients(tf.log(Psa), ws_actor)
        dws_actor = [ tf.placeholder(dw.dtype, dw.get_shape()) for dw in gradients_actor ]
        update_actor = [ tf.assign_add(w, ws) for w, ws in zip(ws_actor, dws_actor) ]
        
        ws_critic = tf.get_collection("critic")
        gradients_critic = tf.gradients(V, ws_critic)
        dws_critic = [ tf.placeholder(dw.dtype, dw.get_shape()) for dw in gradients_critic ]
        update_critic = [ tf.assign_add(w, ws) for w, ws in zip(ws_critic, dws_critic) ]
        

        writer = tf.summary.FileWriter('/logs/' +  model_name)
        saver = tf.train.Saver()

In [53]:
def next_action(actions, get_max=False):
    n = actions.shape[0]
    return choice(n, p=actions) if not get_max else np.argmax(actions)
    
def discount(rewards, y):
    r_accum = 0.0
    for i in reversed(range(len(rewards))):
        r_accum = rewards[i] = rewards[i] + y * r_accum
        
    return rewards
            
            
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess, model_path)
    
    r_total = 0
    
    for t in range(200000):
        k = 20000.0
        lr = 0.5 * k / (k + t)
#         lr = 0.2
        
        _s = env.reset()
        I = 1.0
        
        while True:
            #calculate Ps
            [_ps, _v]= sess.run([Ps, V], feed_dict={s: _s})
            
            #select action
            _a = next_action(_ps)
            
            #take step
            _s1, _r, done, info = env.step(_a)
            r_total += _r
            
            #
            _v1 = sess.run(V, feed_dict={s: _s1})
            
            #train
            [_gradients_actor, _gradients_critic] = sess.run([gradients_actor, gradients_critic], {s: _s, a: _a})
            
            
            _updates_actor = { dw: lr * dp_dw * (_r + y * _v1 - _v) for dw, dp_dw in zip(dws_actor, _gradients_actor)}
            _updates_critic = { dw: lr * dv_dw * (_r + y * _v1 - _v) for dw, dv_dw in zip(dws_critic, _gradients_critic)}
            
            feed_dict = {}
            feed_dict.update(_updates_actor)
            feed_dict.update(_updates_critic)
            
            sess.run(update_actor + update_critic, feed_dict=feed_dict)
            
            #update state
            _s = _s1
#             I *= y

            if done:
                break
            
        

        if t % 500 == 0:
            print(int(r_total), "of", 500)
#             print(t_ward)
            r_total = 0
#             saver.save(sess, model_path)


(0, 'of', 500)
(9, 'of', 500)
(10, 'of', 500)
(4, 'of', 500)
(2, 'of', 500)
(8, 'of', 500)
(9, 'of', 500)
(16, 'of', 500)
(16, 'of', 500)
(11, 'of', 500)
(14, 'of', 500)
(14, 'of', 500)
(21, 'of', 500)
(43, 'of', 500)
(55, 'of', 500)
(85, 'of', 500)
(111, 'of', 500)
(118, 'of', 500)
(122, 'of', 500)
(98, 'of', 500)
(93, 'of', 500)
(108, 'of', 500)
(96, 'of', 500)
(117, 'of', 500)
(105, 'of', 500)
(89, 'of', 500)
(123, 'of', 500)
(97, 'of', 500)
(100, 'of', 500)
(95, 'of', 500)
(108, 'of', 500)
(103, 'of', 500)
(100, 'of', 500)
(122, 'of', 500)
(98, 'of', 500)
(135, 'of', 500)
(115, 'of', 500)
(99, 'of', 500)
(105, 'of', 500)
(111, 'of', 500)
(111, 'of', 500)
(123, 'of', 500)
(120, 'of', 500)
(107, 'of', 500)
(121, 'of', 500)
(101, 'of', 500)
(106, 'of', 500)
(130, 'of', 500)
(123, 'of', 500)
(107, 'of', 500)
(105, 'of', 500)
(124, 'of', 500)
(126, 'of', 500)
(120, 'of', 500)
(196, 'of', 500)
(216, 'of', 500)
(222, 'of', 500)
(199, 'of', 500)
(215, 'of', 500)
(228, 'of', 500)
(232, 'of'

KeyboardInterrupt: 

In [ ]:
tf.gather(Ps, a)

In [47]:
_s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    saver.restore(sess, model_path)
    
    for i in range(1000):
        
        _ps = sess.run(Ps, feed_dict={s: _s})
        _a = next_action(_ps, 0)
        
        
        print _s
        
        print [_psa for _psa in _ps], _a
        
        _s, _r, done, info = env.step(_a)
        env.render()
        
        print _s
        
        print("")

        if done:
            print(_r)
            break

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
4

4
[0.99929619, 0.00069224928, 8.7665958e-06, 2.6709235e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
4

4
[0.99929619, 0.00069224928, 8.7665958e-06, 2.6709235e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
4

4
[0.99929619, 0.00069224928, 8.7665958e-06, 2.6709235e-06] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99920779, 0.00078097288, 9.0313206e-06, 2.1757803e-06] 